# めちゃめちゃ参考文献

https://www.youtube.com/@user-ej3zj6kv9z

そーすこーどとかほぼパクリなので, 使用の際は気を付けて

# import

In [1]:

import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
from my_library.horse import *
import pickle


# path

In [3]:
!ls

Data       main.ipynb my_library


In [4]:
# path_ubu = '/home/hipro/デスクトップ/Hor/Data/20_21'
# 相対パス
path_ = 'Data/'


# load data

In [5]:
results = load_csv(path_+'results.csv')
horse_results = load_csv(path_+'horse_results.csv')
peds = load_csv(path_+'peds.csv')
peds.fillna('nan',inplace=True) # 何回やってもロードすると, nanが出る
return_tables = load_csv(path_+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

# データ形式

### データ　見た目こんな感じ

In [6]:
"""
    レース結果を記録したテーブル
    results.index : Int64Index, index は race_id
    index だけでは一意にレコードが求まらないので, 注意
"""
results.head() # index : race_id

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,course_len,weather,race_type,ground_state,date,horse_id,jockey_id
202005020906,1,4,8,キタノヴィジョン,牡3,56.0,三浦皇成,2:11.9,NaN,3.8,1.0,468(+2),[東] 萱野浩二,2100,曇,ダート,稍重,2020年5月23日,2017102011,1122
202005020906,2,1,1,ジャッジ,牡3,56.0,Ｍ．デム,2:12.0,3/4,7.3,4.0,500(+16),[東] 和田雄二,2100,曇,ダート,稍重,2020年5月23日,2017102704,5212
202005020906,3,6,12,アメリカンフェイス,牡3,56.0,吉田豊,2:12.0,ハナ,8.8,5.0,472(0),[東] 戸田博文,2100,曇,ダート,稍重,2020年5月23日,2017110066,733
202005020906,4,7,13,ミュアウッズ,牝3,54.0,レーン,2:12.6,3.1/2,4.2,2.0,580(-10),[東] 尾関知人,2100,曇,ダート,稍重,2020年5月23日,2017105276,5585
202005020906,5,5,10,ヤマノマタカ,牡3,56.0,石川裕紀,2:13.1,3,22.9,8.0,466(-2),[東] 相沢郁,2100,曇,ダート,稍重,2020年5月23日,2017103008,1150


In [7]:
""" 
    馬の過去の全戦績を記したテーブル
    horse_results.index : Int64Index, index は horse_id
    indexはhorse_id
    こちらも, index だけでは一意にレコードが定まらないので注意
"""
horse_results.head()

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2017105800,2021/07/08,名古屋,曇,7.0,C12組,NaN,10.0,4.0,4.0,6.6,...,NaN,**,4-3,0.0-39.5,NaN,479(-1),NaN,NaN,ナツミフルーリー,NaN
2017105800,2021/06/30,名古屋,曇,5.0,C14組,NaN,10.0,3.0,3.0,12.2,...,0.7,**,5-6-5,0.0-40.3,39.4,480(-4),NaN,NaN,マルカブレーブ,10.2
2017105800,2021/06/16,名古屋,雨,5.0,美穂ちゃんバースデイ,NaN,11.0,2.0,2.0,13.5,...,2.7,**,10-10-8,0.0-38.2,38.4,484(+3),NaN,NaN,タイキスピネル,2.6
2017105800,2021/06/03,名古屋,晴,6.0,C14組,NaN,10.0,8.0,9.0,3.8,...,1.0,**,3-4-4,0.0-39.6,40.1,481(-6),NaN,NaN,トーホウボッカ,NaN
2017105800,2021/05/19,名古屋,雨,5.0,C15組,NaN,11.0,8.0,11.0,29.8,...,0.5,**,3-3-3,0.0-38.4,38.6,487(0),NaN,NaN,キングリアリティー,10.2


In [8]:
""" 
    馬の血統データ
    peds.index : Int64Index, index は horse_id
    カタカナ, 英字が混じっているので, 注意
"""
peds.head() # 5世代分の血統データ

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,...,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61
2017100590,クロフネ,ハッピーティア,フレンチデピュティ,ブルーアヴェニュー,ホワイトマズル,ハッピーチケット,DeputyMinister,Mitterand,ClassicGoGo,ElizaBlue,...,Tyrant,FlightTable,Halo,WishingWell,Lyphard,Prodice,Herbager,SilverCoin,チャイナロック,ソシアルバターフライ
2017106441,スピルバーグ,ミュージー,ディープインパクト,プリンセスオリビア,ElPrado,Foible,サンデーサイレンス,ウインドインハーヘア,Lycius,DanceImage,...,TomFool,Ghazni,Nasrullah,Lalun,PrinceJohn,NileLily,Turnto,YourGame,RestlessNative,EnchantedEve
2017106225,ケープブランコ,ヒラボクキャロル,Galileo,LaurelDelight,ダンスインザダーク,チョウカイキャロル,SadlersWells,アーバンシー,Presidium,Foudroyer,...,KeytotheMint,NativePartner,HailtoReason,Bramalea,Graustark,GoldenTrail,RaiseaNative,GoldDigger,VaguelyNoble,SleekDancer
2014105693,キンシャサノキセキ,マイワイルドローズ,フジキセキ,ケルトシャーン,WildAgain,RoseofMull,サンデーサイレンス,ミルレーサー,PleasantColony,Featherhill,...,Dante,Clovelly,SwordDancer,Kerala,Buckpasser,Intriguing,Ribot,Pocahontas,DarkStar,Vulcania
2013101632,メイショウサムソン,シルクヴァルキリー,オペラハウス,マイヴィヴィアン,マーベラスサンデー,シルクフレアー,SadlersWells,Colorspin,ダンシングブレーヴ,ウイルプリンセス,...,LaughAloud,HoldMeClose,Nearctic,Natalma,AdmiralsVoyage,Petitioner,RaiseaNative,SweetTooth,ノノアルコ,Mortefontaine


In [13]:
return_tables.index

Int64Index([202001010101, 202001010101, 202001010101, 202001010101,
            202001010101, 202001010101, 202001010101, 202001010102,
            202001010102, 202001010102,
            ...
            202201020611, 202201020611, 202201020612, 202201020612,
            202201020612, 202201020612, 202201020612, 202201020612,
            202201020612, 202201020612],
           dtype='int64', length=70985)

In [12]:
""" 
    return_tables レースの払い戻し金額が記されたテーブル
    return_tables.index : Int64Index, index は race_id
    こちらも, index では一意にレコードが定まらない
    カラムの
        0 : 払い戻し種別
        1 : 馬番, 
            ex) 6 なら 馬番6が払い戻し対象
        2 : 払い戻し金額 (単位 : 円, 100円賭けた時の払い戻し金額), 
            ex) 複勝で カラム(2) が 490br130 なら, 馬番6の払い戻しが 490円 馬番2が130円
        3 : 多分人気度を表してる, 使ってないから dont care
"""
return_tables.head()

,0,1,2,3
202001010101,単勝,6,1600,3
202001010101,複勝,6br2,490br130,4br2
202001010101,馬連,2 - 6,1410,4
202001010101,ワイド,2 - 6br3 - 6br2 - 3,150br170br110,3br4br1
202001010101,馬単,6 → 2,3920,9


# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [7]:
ll = LearnLGBM(peds,results,horse_results)
ll.learn_lgb()

  0%|          | 0/18799 [00:00<?, ?it/s]

pe finish
pe regularizrd


  0%|          | 0/279 [00:00<?, ?it/s]

  0%|          | 0/279 [00:00<?, ?it/s]

  0%|          | 0/279 [00:00<?, ?it/s]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 76276
[LightGBM] [Info] Number of data points in the train set: 97724, number of used features: 172


In [8]:
sl = Simulater(ll.model)

race_id_list = list(set(ll.x_test.index))
data_c = ll.r.data_c.loc[ll.x_test.index[0]:]

all_results = sl.get_result_df(data_c, return_tables)



In [15]:
calc_tansho(all_results,odds_alpha=1)

的中率 0.006996770721205597
収益   -2500.0 円
race dist {'01': 0, '02': 0, '03': 0, '04': 2, '05': 1, '06': 2, '07': 4, '08': 1, '09': 1, '10': 0, '11': 2, '12': 0}


# grid search

In [ ]:
train_data = ll.r.data_c
train, valid = split_data(train_data,test_size=0.2,rank_learning=False)
# 0.8 : 0.2

x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_valid = valid.drop(['rank', 'date','単勝'], axis=1)
y_valid = valid['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
valid_query = x_valid.groupby(x_valid.index).size()
valid = lgb.Dataset(x_valid, y_valid, reference=train, group=valid_query)


def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用

    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        'num_iterations': hp.choice('num_iterations',range(50,120)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 25      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)


trials = Trials()
optimize(trials)


# milestone

In [ ]:
{'lambdarank_truncation_level': 23, 'learning_rate': 0.04425892924779881, 'num_iterations': 1}

In [19]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 24,
                'learning_rate':0.04425892924779881,
                'num_iterations': 51,
                # 'lambdarank_truncation_level': 10,
                # 'learning_rate': 0.02273417953255777,
                # n_estimators は num_iterations と同義　default = 100
                # 'n_estimators': 97,
                # 'num_leaves': 42,
                'force_col_wise':True
            }



In [161]:
import warnings

warnings.filterwarnings("ignore")

ll.learn_lgb2(train,lgbm_params=lgbm_params)

[LightGBM] [Info] Total Bins 76081
[LightGBM] [Info] Number of data points in the train set: 96940, number of used features: 172


# race_id_list

In [40]:
# 今週
race_id_list = ['2022040308{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022100408{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022010208{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022040307{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022100407{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022010207{}'.format(str(i).zfill(2)) for i in range(1,13)]



# race_id_list += ['2022040306{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022100406{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022010206{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090303{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090304{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022020103{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020104{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020105{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020106{}'.format(str(i).zfill(2)) for i in range(1,13)]



# Predictor

In [21]:
pt = Predictor(peds,results,horse_results,race_id_list)
# ********* test_size = 0  : ずっと0にしてなかった...
pt.learn_lgb(lgbm_params=lgbm_params,test_size=0)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/18869 [00:00<?, ?it/s]

pe finish
pe regularizrd


...

  0%|          | 0/279 [00:00<?, ?it/s]

  0%|          | 0/279 [00:00<?, ?it/s]

  0%|          | 0/279 [00:00<?, ?it/s]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 80424
[LightGBM] [Info] Number of data points in the train set: 121733, number of used features: 172


# 新潟

In [367]:
base  = '2022040308'
pt.predict(base+'07')


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

              馬番    scores
202204030807  14 -0.009576
202204030807   9 -0.409627
202204030807  13 -0.532311
202204030807  15 -0.762949
202204030807   5 -0.815435
202204030807   7 -0.906838
202204030807   4 -0.914260
202204030807  10 -0.931911
202204030807   1 -1.132334
202204030807   3 -1.184594
202204030807  12 -1.283133
202204030807  11 -1.323496
202204030807   2 -1.442930
202204030807   8 -2.367764
202204030807   6 -2.421679


# 小倉

In [365]:
base  = '2022100408'
        #  202210040802 
# for i in ['07','08','09','10','11']:
pt.predict(base+'07')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

              馬番    scores
202210040807  16  0.346642
202210040807  15  0.307885
202210040807   9 -0.091335
202210040807   7 -0.225866
202210040807  11 -0.227937
202210040807   8 -0.343088
202210040807   5 -0.718924
202210040807  12 -1.036220
202210040807  10 -1.153685
202210040807   6 -1.253014
202210040807   2 -1.313656
202210040807  14 -1.360719
202210040807   4 -1.400164
202210040807  13 -1.594319
202210040807   1 -1.810442
202210040807   3 -2.208708


# 札幌

In [364]:
base  = '2022010208'
pt.predict(base+'07')


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

              馬番    scores
202201020807   1 -0.042919
202201020807   8 -0.153212
202201020807  11 -0.487057
202201020807   2 -0.519089
202201020807  14 -0.530410
202201020807   3 -0.704115
202201020807   5 -0.816080
202201020807   6 -0.878614
202201020807  13 -0.924903
202201020807  12 -1.397026
202201020807   9 -1.412963
202201020807  10 -1.533956
202201020807   4 -1.560519
202201020807   7 -1.677366


# 当日の収支計算

return_tables　と, return_tables_todayで形式が異なるので, 形式をそろえる

# 当日 sim 手順

In [22]:
data =  ShutubaTable.scrape(race_id_list, pt.date)
st = ShutubaTable(data)
st.preprocessing()
st.merge_horse_results(pt.hr)
st.merge_peds(pt.pe.peds_cat)
st.process_categorical(pt.r.le_horse, pt.r.le_jockey, pt.r.data_pe)


tsl = TodaySimulater(pt.model)
tsl.return_table_today(race_id_list)
all_results = tsl.get_result_df( st.data_c, tsl.return_tables,race_id_list, kaime='all',odds=1)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

In [41]:
all_results

,pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb
202204030801,"[13, 5, 9, 8, 10, 4, 1, 2, 11, 12, 3, 7, 6]","[10, 8, 9]",3.5,"[1.7, 3.3, 3.5]",21.9,"[5.8, 7.6, 14.2]",32.6,64.5,259.7,"[[8, 10], [9, 10], [8, 9]]"
202204030802,"[4, 16, 14, 2, 8, 10, 11, 3, 13, 17, 7, 9, 5, ...","[10, 3, 14]",5.3,"[1.9, 7.2, 2.4]",79.5,"[20.8, 6.6, 33.8]",112.0,160.3,976.7,"[[3, 10], [10, 14], [3, 14]]"
202204030803,"[3, 8, 6, 12, 1, 15, 2, 11, 7, 9, 14, 10, 13, ...","[3, 1, 6]",5.4,"[2.0, 8.1, 9.9]",80.1,"[23.0, 31.9, 104.9]",106.8,649.2,2468.9,"[[1, 3], [3, 6], [1, 6]]"
202204030804,"[11, 1, 10, 13, 16, 6, 12, 5, 8, 2, 14, 17, 3,...","[3, 13, 17]",51.1,"[13.5, 8.6, 8.8]",908.8,"[171.7, 116.9, 77.5]",1698.1,4495.3,32267.6,"[[3, 13], [3, 17], [13, 17]]"
202204030805,"[3, 9, 7, 4, 12, 8, 10, 13, 1, 6, 11, 5, 2]","[6, 4, 1]",3.8,"[1.7, 4.8, 4.2]",28.8,"[12.2, 11.5, 22.9]",40.2,127.1,459.4,"[[4, 6], [1, 6], [1, 4]]"
202204030806,"[15, 17, 18, 7, 16, 12, 9, 8, 10, 1, 14, 13, 1...","[17, 10, 12]",2.5,"[1.4, 8.3, 3.4]",48.4,"[18.6, 7.5, 55.7]",69.4,151.8,535.7,"[[10, 17], [12, 17], [10, 12]]"
202204030807,"[14, 13, 4, 15, 7, 5, 10, 9, 11, 12, 1, 3, 2, ...","[14, 7, 10]",2.5,"[1.4, 2.2, 5.7]",9.5,"[4.5, 11.3, 22.3]",14.8,56.7,175.4,"[[7, 14], [10, 14], [7, 10]]"
202204030808,"[13, 4, 7, 10, 11, 5, 9, 6, 8, 12, 1, 14, 2, 1...","[9, 4, 7]",15.5,"[5.5, 3.5, 8.3]",40.1,"[10.3, 23.8, 11.0]",128.6,168.1,1254.6,"[[4, 9], [7, 9], [4, 7]]"
202204030809,"[8, 7, 10, 2, 13, 1, 12, 14, 4, 9, 6, 3, 11, 5]","[12, 1, 2]",4.6,"[1.9, 2.4, 3.3]",17.7,"[7.8, 10.4, 11.8]",35.7,56.6,255.1,"[[1, 12], [2, 12], [1, 2]]"
202204030810,"[1, 14, 5, 12, 6, 9, 10, 15, 3, 13, 11, 4, 2, ...","[10, 14, 5]",22.4,"[6.9, 8.2, 6.0]",187.3,"[51.3, 46.2, 58.2]",526.3,1058.8,6427.2,"[[10, 14], [5, 10], [5, 14]]"


In [24]:
calc_tansho(all_results,is_today=True)

的中率 0.05555555555555555
収益   -2810.0 円
race dist {'01': 0, '02': 0, '03': 1, '04': 0, '05': 0, '06': 0, '07': 1, '08': 0, '09': 0, '10': 0, '11': 0, '12': 0}


# previous params

In [16]:
all_results


,pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb
202204030501,"[15, 8, 14, 12, 2, 7, 9, 10, 6, 4, 11, 3, 1, 5...","[15, 8, 12]",3.0,"[1.2, 1.1, 2.2]",2.8,"[1.6, 4.7, 3.5]",7.3,8.3,38.0,"[[8, 15], [12, 15], [8, 12]]"
202204030502,"[1, 15, 7, 13, 3, 14, 5, 10, 8, 11, 6, 2, 12, ...","[15, 5, 1]",5.8,"[1.8, 6.8, 1.3]",88.7,"[25.0, 3.0, 14.0]",170.0,50.2,435.9,"[[5, 15], [1, 15], [1, 5]]"
202204030503,"[1, 7, 12, 9, 15, 5, 13, 14, 2, 4, 11, 3, 10, 8]","[7, 9, 14]",6.9,"[2.1, 4.4, 3.2]",58.4,"[16.5, 8.9, 25.5]",109.5,164.0,942.7,"[[7, 9], [7, 14], [9, 14]]"
202204030504,"[3, 11, 10, 7, 6, 1, 4, 9, 8, 2, 5, 13, 14, 12]","[10, 13, 9]",5.6,"[2.0, 6.3, 1.8]",82.4,"[28.2, 5.6, 17.8]",124.4,93.0,706.4,"[[10, 13], [9, 10], [9, 13]]"
202204030505,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[10, 2, 7]",7.5,"[1.5, 1.8, 1.1]",31.1,"[6.4, 2.5, 3.0]",58.0,9.3,106.7,"[[2, 10], [7, 10], [2, 7]]"
202204030506,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[2, 6, 11]",6.6,"[1.9, 2.5, 1.4]",42.2,"[12.3, 4.9, 5.5]",69.0,40.0,266.0,"[[2, 6], [2, 11], [6, 11]]"
202204030507,"[18, 14, 15, 12, 16, 5, 13, 11, 10, 4, 3, 7, 2...","[12, 16, 4]",6.5,"[2.0, 1.8, 2.2]",19.9,"[6.8, 7.2, 7.5]",32.6,39.6,247.6,"[[12, 16], [4, 12], [4, 16]]"
202204030508,"[1, 12, 8, 15, 9, 7, 13, 11, 4, 5, 10, 6, 14, ...","[9, 13, 12]",4.6,"[2.0, 5.8, 1.7]",69.7,"[20.4, 3.9, 13.2]",94.9,57.9,473.8,"[[9, 13], [9, 12], [12, 13]]"
202204030509,"[14, 10, 6, 9, 4, 7, 11, 8, 12, 13, 2, 5, 1, 3...","[8, 13, 6]",1.7,"[1.1, 4.6, 1.4]",23.8,"[8.1, 2.1, 18.7]",29.8,32.5,136.6,"[[8, 13], [6, 8], [6, 13]]"
202204030510,"[2, 7, 1, 13, 5, 15, 11, 12, 4, 6, 8, 3, 10, 9...","[13, 11, 5]",3.8,"[1.5, 1.9, 2.2]",11.3,"[4.5, 5.8, 8.3]",19.8,29.9,123.7,"[[11, 13], [5, 13], [5, 11]]"


# 日付に注意

In [47]:
date = '2022/12/31'

# Results scraping

In [ ]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

# Horse_results scraping

In [ ]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)

# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

  0%|          | 0/957 [00:00<?, ?it/s]

# Peds scraping

In [ ]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

  0%|          | 0/957 [00:00<?, ?it/s]

  0%|          | 0/957 [00:00<?, ?it/s]

# Return scraping

In [ ]:
returns_tmp = Return.scrape(race_id_list)


  0%|          | 0/72 [00:00<?, ?it/s]

# update

In [ ]:
new_results = update_data(load_csv(path_win+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_win+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_win+'peds.csv'),peds_tmp)
returns = load_csv(path_win+'return.csv')
returns.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
returns_tmp.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
new_return = update_data(returns,returns_tmp)


# save

In [ ]:
new_results.to_csv(path_win+'results.csv')
new_horse_results.to_csv(path_win+'horse_results.csv')
new_peds.to_csv(path_win+'peds.csv')
new_return.to_csv(path_win+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習
